# Описание блокнота

Данный блокнот служит для наглядного тестирования ядра парсера.

# 0. Импорт библиотек

In [1]:
# !pip freeze > jupiter-requirements.txt

In [2]:
from CoolRespProject.modules_parser import cr_additional as cra  # Хранилище побочных функций
from CoolRespProject.modules_parser import cr_defaults   as crd  # Константы парсера
from CoolRespProject.modules_parser import cr_exceptions as cre  # Исключения парсера
from CoolRespProject.modules_parser import cr_reader     as crr  # Взаимодействие с Excel
from CoolRespProject.modules_parser import cr_parser     as crp  # Регулярный парсер
from CoolRespProject.modules_parser import cr_writter    as crw  # Форматная запись в Excel
import os
import re

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 905)

In [4]:
from IPython.core.interactiveshell import InteractiveShell
jup_conds = ['all',                # Показывать все вызовы
             'last',               # Показывать последнюю общность вызовов
             'last_expr',          # Показывать самый последний вызов
             'none',               # Ничего не показывать
             'last_expr_or_assign' # Показывать последний вызов или присваивание
            ]

InteractiveShell.ast_node_interactivity = jup_conds[0]

In [5]:
base_dir = os.getcwd()
path_load = os.path.join(base_dir, 'Tests', 'Datasets')  # Путь загрузки обрабатываемых файлов
path_save = os.path.join(base_dir, 'Tests', 'Results')   # Путь сохранения итогов обработки
path_json = os.path.join(base_dir, 'Tests', 'Jsons')     # Путь сохранения json файлов
path_best = os.path.join(base_dir, 'Tests', 'JSBest')    # Где хранятся проверенные вручную json файлы

# 1. Тестирование модуля парсинга

In [6]:
help(crp)

Help on module CoolRespProject.modules_parser.cr_parser in CoolRespProject.modules_parser:

NAME
    CoolRespProject.modules_parser.cr_parser - Парсинг датафрейма с ячейками расписания конкретной группы.

FUNCTIONS
    expand_dates(dates: 'Список подстрок с датами предмета', year: 'Год (берётся из периода расписания)', day: 'День недели, соответствующий списку дат') -> 'Список объектов дат'
        Функция абсолютного разбития дат
    
    format_group(group: 'Подстрока форматируемой подгруппы') -> 'Отформатированная подгруппа'
        Функция форматирования подгруппы
    
    format_prep(lecturer: 'Форматируемая подстрока преподавателя') -> 'Отформатированная подстрока преподавателя'
        Функция форматирования записи преподавателя
    
    format_tip(tip: 'Подстрока форматируемого типа пары') -> 'Отформатированный тип пары'
        Функция форматирования типа пары
    
    parser(stuff: 'База обработки', timey: 'Период расписания', year: 'Год расписания') -> 'База парсинга'
      

In [7]:
# Конвертация базы парсинга в датафрейм
def parse_convert_to_dataframe(bd_parse):
    df = pd.DataFrame(data=bd_parse,
                      columns=['day',        # День недели
                               'num',        # Номер пары
                               'item_name',  # Название предмета
                               'teacher',    # Препод
                               'type',       # Тип пары
                               'pdgr',       # Подгруппа
                               'date_pair',  # Дата
                               'cab'         # Кабинет
                              ]).explode('date_pair').reset_index(drop=True).drop_duplicates()

    # Выделение реального периода расписания
    date_min = df['date_pair'].dropna().min().strftime('%d.%m.%Y')
    date_max = df['date_pair'].dropna().max().strftime('%d.%m.%Y')

    # Имя датафрейма
    df.name = f'Респа для {group} на [{date_min} - {date_max}]'
    
    return df

In [8]:
count_act_df = count_saved_df = count_best_df = difs_cab_count = difs_all_count = 0
save_difs = []
dataframes = []
# Выбор подгрупп
g2, g3 = '0', '0'

# Проход по всем файлам и подпапкам в директории
for root, d, files in os.walk(path_load):
    for file in files:    
        # Построение пути к заданному файлу в заданной директории
        file_path = os.path.join(path_load, file)
        print(f'Открытие файла «{file_path}»')

        try:
            book = crr.read_book(file_path)
            sheets = crr.see_sheets(book)
            for sheet_name in sheets:
                sheet = crr.take_sheet(book, sheet_name)
                sheet_info = crr.group_choice(sheet)
                period, year, groups, start_end = sheet_info.values()

                for group in groups:
                    # Создать расписание для каждой группы
                    bd_process = crr.prepare(sheet, group, start_end)

                    # Создание базы с запарсенными данными
                    df = crp.parser(bd_process, period, year)
                    dataframes.append(df)

                    f_book = crw.create_resp(df, g2, g3, t_yn=True, p_yn=True)
                    crw.save_resp(f_book, f'Tests/Results/{cra.create_name(df)}.xlsx')

                    # Сохранение базы парсинга в JSON
                    cra.dataframe_to_json(df, path_json)
                    # cra.database_to_json(df, path_json)
                    df.name = cra.create_name(df)

                    # Путь к сохранённой базе парсинга
                    path_to_act = os.path.join(path_json, f'{df.name}.json')
                    # Путь к "идеальной" версии этой базы (если есть)
                    path_to_best_if = os.path.join(path_best, f'{df.name}.json')
                    # Если у базы парсинга есть "идеальная" версия, то сравнить их
                    if os.path.exists(path_to_best_if):
                        # Количество записей в запарсенном датафрейме
                        count_act_df += df.shape[0]

                        # Открытие запарсенного датафрейма, сохранённого в файл (на случай ошибок в сохранении)
                        df1 = pd.read_json(path_to_act)
                        df1['date_pair'] = pd.to_datetime(df1['date_pair']).dt.strftime('%Y-%m-%d')
                        count_saved_df += df1.shape[0]

                        # Открытие "идеального" датафрейма, сохранённого в файл
                        df2 = pd.read_json(path_to_best_if).explode('date_pair').reset_index(drop=True)
                        df2['date_pair'] = pd.to_datetime(df2['date_pair']).dt.strftime('%Y-%m-%d')
                        count_best_df += df2.shape[0]

                        # Сравнение датафреймов
                        difs = cra.df_differences(df1, df2)
                        if not difs.empty:
                            difs_all_count += difs[difs['_merge'] == 'right_only'].shape[0]
                            # save_difs.append(difs.merge(bd_process, how='left'))
                            # save_difs[-1].name = group

                        # Сравнение без учёта столбца кабинетов
                        # Открытие запарсенного датафрейма, сохранённого в файл (на случай ошибок в сохранении)
                        df1 = pd.read_json(path_to_act).drop(['cab'], axis=1)
                        df1['date_pair'] = pd.to_datetime(df1['date_pair']).dt.strftime('%Y-%m-%d')

                        # Открытие "идеального" датафрейма, сохранённого в файл
                        df2 = pd.read_json(path_to_best_if).drop(['cab'], axis=1).explode('date_pair').reset_index(drop=True)
                        df2['date_pair'] = pd.to_datetime(df2['date_pair']).dt.strftime('%Y-%m-%d')

                        # Сравнение датафреймов
                        difs = cra.df_differences(df1, df2)
                        if not difs.empty:
                            difs_cab_count += difs[difs['_merge'] == 'right_only'].shape[0]
                            save_difs.append(difs.merge(bd_process, how='left'))
                            save_difs[-1].name = group
                    else:                    
                        # файл не существует
                        print(f'Лучшего результата нет ({df.name})')
                    df.insert(0, "group", group)
        except Exception as err:
            print(err)
        finally:
            print()


print(f'\nВсего выявлено: {count_act_df} записей  ')
print(f'Всего сохранено: {count_saved_df} записей  ')
print(f'Всего должно быть: {count_best_df} записей  ')

print(f'\nВсего отличий: {difs_all_count - difs_cab_count} записей отличаются от правильных кабинетами  ')
print(f'Всего отличий: {difs_cab_count} записей отличаются от правильных чем-то ещё  ')

diffs_full = difs_all_count / count_best_df
diffs_cabs = (difs_all_count  / count_best_df) - (difs_cab_count / count_best_df)
print(f'\nПроцент коррапта: {diffs_cabs:.2%} из-за кабинетов  ')
print(f'Процент коррапта: {diffs_full - diffs_cabs:.2%} из-за прочего  ')

print(f'\nТочность: {1 - diffs_cabs:.2%} с учётом кабинетов  ')
print(f'Точность: {1 - (diffs_full - diffs_cabs):.2%} без учёта кабинетов  ')

Открытие файла «E:\CoolResp\Tests\Datasets\1_KURS_2018-2019_2semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\2_kurs_2semestr_2018-2019.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\3_kurs_2018-2019_II_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\3_kurs_2019-2020_II_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\4 курс ОЕ, МЕ-81Б ноябрь, декабрь 2021.xlsx»
Лучшего результата нет (Респа для МЕ-81б на [01.11.2021 - 29.12.2021])
Лучшего результата нет (Респа для ОЕ-81б на [01.11.2021 - 28.12.2021])

Открытие файла «E:\CoolResp\Tests\Datasets\4_kurs_2018-2019_II_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\921_2_semestr_2019-2020.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\IT-81b_ME-81b_OE-81b_2020-2021_2_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\ME-71b_OE-71b_IT-71b_2_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\MITE-91_2019-2020_II_semestr.xls»

Открытие файла «E:\CoolResp\Tests\Datasets\MIVT-91_2019-2020_II_sem

Всего отличий: 144 записей отличаются от правильных кабинетами  
Всего отличий: 274 записей отличаются от правильных чем-то ещё  

Процент коррапта: 1.49% из-за кабинетов  
Процент коррапта: 2.83% из-за прочего  

<div class="alert alert-info">
<font size="5"><b>Итог эталонной сверки результатов парсинга</b></font>

Всего выявлено: 9601 записей  
Всего сохранено: 9601 записей  
Всего должно быть: 9694 записей  

Всего отличий: 144 записей отличаются от правильных кабинетами  
Всего отличий: 198 записей отличаются от правильных чем-то ещё  

Процент коррапта: 1.49%  из-за кабинетов  
Процент коррапта: 2.04%  из-за прочего  

Точность: 98.51%  с учётом кабинетов  
Точность: 97.96%  без учёта кабинетов  

</div>

In [9]:
bad_files = ['D:\CoolResp\Tests\Datasets\Изменения_МИВТ-11 2021-2022  I семестр.xls']

# Проход по всем файлам в чёрном списке
for file in bad_files:    
    # Построение пути к заданному файлу в заданной директории
    file_path = os.path.join(path_load, file)
    print(f'Открытие файла «{file_path}»')

    try:
        book = crr.read_book(file_path)
        sheets = crr.see_sheets(book)
        for sheet_name in sheets:
            sheet = crr.take_sheet(book, sheet_name)
            sheet_info = crr.group_choice(sheet)
            period, year, groups, start_end = sheet_info.values()

            for group in groups:
                # Создать расписание для каждой группы
                bd_process = crr.prepare(sheet, group, start_end)

                # Создание базы с запарсенными данными
                df = crp.parser(bd_process, period, year)
    except Exception as err:
        print(err)
    finally:
        print()

Открытие файла «D:\CoolResp\Tests\Datasets\Изменения_МИВТ-11 2021-2022  I семестр.xls»
[Errno 2] No such file or directory: 'D:\\CoolResp\\Tests\\Datasets\\Изменения_МИВТ-11 2021-2022  I семестр.xls'



In [10]:
len(save_difs)

23

In [11]:
len(save_difs)

23

In [12]:
#### Без-кабинетники
# 00: Перепроверить. Возможно, парсер "потерял" часть предметов (либо "идеальные" результаты избыточны, непонятно)
# 01: Потеря пар (???)
# 02: Ошибочное распознание типов пар
# 03: Возможны проблемы тестового вывода, проверить вручную
# 04: Потеря пар (???)
# 05: Потеря пар из-за отсутствия отступа после "с ... по ... г."
# 06: См. №5 (общий документ)
# 07: Появление лишних пар (возможно, из-за кривых дат)
# 08: Возможны проблемы тестового вывода, проверить вручную (даты)
# 09: NaN в дате вида "4.02.21г.", подозрительные записи
# 10: См. №9 (общий документ)
# 11: См. №9 (общий документ)
# 12: Ошибочное разделение пар "даты-предмет" из-за кривой семантики учебки (мб они были бухими)
# 13: Потеря пар из-за дат вида "8.10.21г."
# 14: См. №13 (общий документ)
# 15: Возможны проблемы тестового вывода, проверить вручную
# 16: См. №15 (общий документ)
# 17: Появление лишних пар (возможно, из-за кривых дат)
# 18: Потеря пар из-за кривого обозначения общих подгрупп (обычно они не пишутся, но здесь указаны как "1,2п/гр")
# 19: Ошибки в "идеальных" результатах, исправить документ
n_dif = 19
save_difs[n_dif].name
save_difs[n_dif]

'ПЕ-61б'

,day,num,item_name,teacher,type,pdgr,date_pair,_merge,rec,cabs
0,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-02-26,left_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
1,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-03-05,left_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
2,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-03-12,left_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
3,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-03-19,left_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
4,ПТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-01-04,left_only,"Технологии программирования; с 01.02.19г. по 15.02.19г. - теория; 22.02,01.03.19г. - практика 2 п/гр: доцент Обвинцев О.А.","307 УК№1; ; 310 УК№1, ."
5,ПТ,1,Технологии программирования,доцент Обвинцев О.А.,ПР,2,2019-01-25,left_only,"Технологии программирования; с 01.02.19г. по 15.02.19г. - теория; 22.02,01.03.19г. - практика 2 п/гр: доцент Обвинцев О.А.","307 УК№1; ; 310 УК№1, ."
6,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ЛБ,2,2019-02-26,right_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
7,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ЛБ,2,2019-03-05,right_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
8,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ЛБ,2,2019-03-12,right_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программирования; с 05.02.19г. по 19.02.19г. - теория; 26.02; 05,12,19.03.19г. - лаб.раб. 2 п/гр: доцент Обвинцев О.А. 26.03.19г. Элективные дисцилины по физической культуре; практика: ст.преподаватель Кузнецов В.И.",207 УК№1; ; 312 УК№1; ; 311 УК№1; ; ФЗК_зал
9,ВТ,1,Технологии программирования,доцент Обвинцев О.А.,ЛБ,2,2019-03-19,right_only,"Теория надежности систем; 29.01.19г. - теория; 21,28.05.19г. - лаб.раб. 1 п/гр: ст.преподаватель Белкина А.В. Технологии программ

In [13]:
repr(save_difs[2].iloc[0])

'day                                                                                                                      СР\nnum                                                                                                                       3\nitem_name                                                                                        Теория электрических цепей\nteacher                                                                                       преподаватель Овчинников Д.А.\ntype                                                                                                                     ЛБ\npdgr                                                                                                                    общ\ndate_pair                                                                                                        2019-02-20\n_merge                                                                                                           right_only\

In [14]:
repr(save_difs[n_dif].iloc[0]['item_name'])

"'Технологии программирования'"

In [15]:
repr(save_difs[n_dif].iloc[15]['item_name'])

IndexError: single positional indexer is out-of-bounds

In [ ]:
len(dataframes)

In [ ]:
full_df = pd.concat(dataframes, ignore_index=True)
full_df.shape

In [ ]:
full_df['teacher'].value_counts()

In [ ]:
# full_df[full_df['teacher'].str.contains("Белкина")]['item_name'].value_counts()
full_df[full_df['teacher'].str.contains("доцент")]

In [ ]:
df

In [ ]:
# grp = 'Респа для 921 на [13.01.2020 - 05.06.2020]'
grp = 'Респа для ПЕ-81б на [05.03.2021 - 04.06.2021]'
df = crs.json_to_database(f'E:/CoolResp/Tests/JSBest/{grp}.json')

In [ ]:
df.info()

In [ ]:
df[df['teacher'].str.contains('Фадеев')].sort_values(by=['type', 'pdgr', 'date_pair']).set_index(['type', 'pdgr'])[['date_pair', 'day', 'num', 'teacher', 'item_name']]

In [ ]:
df[df['teacher'].str.contains('Кислицын')].sort_values(by=['type',
                                                           'pdgr',
                                                           'date_pair']).set_index(['type', 'pdgr'])[['date_pair', 'day', 'num', 'teacher', 'item_name']]

In [ ]:
""" Замена всех порченных типов пары на аналоги """
df = crs.replace_type(df, crs.find_bad_type(df))

In [ ]:
df.head(12)

In [ ]:
df = crs.take_data(df, '0', '0').dropna(subset=['date_pair'])
df.head()

In [ ]:
df['item_name'].unique()
df['item_name'].apply(lambda item_name: crs.format_item_name(item_name)).unique()

In [ ]:
df['teacher'].unique()
df['teacher'].apply(lambda teacher: crs.format_teacher(teacher)).unique()

In [ ]:
df['pdgr'].unique()
df['pdgr'].apply(lambda pdgr: crs.format_pdgr(pdgr)).unique()

In [ ]:
df['cab'].unique()
df['cab'].apply(lambda cab: crs.format_cab(cab)).unique()

In [ ]:
string_1 = 'общ'
string_2 = 'общ'

In [ ]:
# %%time
# # Получение объекта форматированного расписания
# f_book = crw.create_resp(df, '0', '0')
# # Сохранение форматированного расписания
# crw.save_resp(f_book, f'E:/CoolResp/Tests/Results/{grp}.xlsx')

In [ ]:
dft = crs.take_data(df, '1', '3')
dft['pdgr'].value_counts()

In [ ]:
# date_max.day_name()

In [ ]:
# date_min.dayofweek